In [2]:
from os import listdir
from os.path import isfile, join
mypath_f = "../data/Wrist Fracture/Fracture"
fractures = [f for f in listdir(mypath_f) if isfile(join(mypath_f, f))]

In [3]:
import pandas as pd
df_f = pd.DataFrame({"path":fractures,"category":["1"]*len(fractures)})
df_f

,path,category
0,1 Male (A View).jpg,1
1,1 Male (L View).jpg,1
2,10 Male (A View).jpg,1
3,10 Male (L View).jpg,1
4,11 Male (A View).jpg,1
...,...,...
106,92 Male (L View).jpg,1
107,93 F (A View).jpg,1
108,93 F (L View).jpg,1
109,94 F (A View).jpg,1


In [4]:
mypath_n = "../data/Wrist Fracture/Normal"
normals = [f for f in listdir(mypath_n) if isfile(join(mypath_n, f))]
df_n = pd.DataFrame({"path":normals,"category":["0"]*len(normals)})
df_n

,path,category
0,1.jpg,0
1,10.jpg,0
2,11.jpg,0
3,12.jpg,0
4,13.jpg,0
...,...,...
77,8.jpg,0
78,80.jpg,0
79,81.jpg,0
80,82.jpg,0


In [5]:
df = pd.concat([df_n,df_f])
df

,path,category
0,1.jpg,0
1,10.jpg,0
2,11.jpg,0
3,12.jpg,0
4,13.jpg,0
...,...,...
106,92 Male (L View).jpg,1
107,93 F (A View).jpg,1
108,93 F (L View).jpg,1
109,94 F (A View).jpg,1


In [25]:
#df.to_csv("../data/Wrist Fracture/allLabels.csv",index=False)

In [6]:
df2 = pd.read_csv("../data/Wrist Fracture/allLabels.csv")
df2

,path,category
0,1.jpg,0
1,10.jpg,0
2,11.jpg,0
3,12.jpg,0
4,13.jpg,0
...,...,...
188,92 Male (L View).jpg,1
189,93 F (A View).jpg,1
190,93 F (L View).jpg,1
191,94 F (A View).jpg,1


In [31]:
idx = [1,2,3,4,5]
img_labels = pd.read_csv("../data/Wrist Fracture/allLabels.csv")
label = img_labels.iloc[idx, 1]
type(label)

pandas.core.series.Series

## Train Test split
This part will split a folder into a stratified Train Test Split. Also, it will take the csv file and split it as well.

In [16]:
total_inputs = len(normals) + len(fractures)
print(f"there are a total of {total_inputs} files \n")
print(f"the distribution is {len(normals)/total_inputs*100:.3f} for normal and {len(fractures)/total_inputs*100:.3f} for fractures")

there are a total of 193 files 

the distribution is 42.487 for normal and 57.513 for fractures


In [14]:
import torch
import numpy as np

l1 = np.array([1,2,3]).tolist()
l2 = np.array([2,3,4,4])


In [15]:
l1

[1, 2, 3]

## Reading Chest Data

In [ ]:
#remove all data with more than one observation
import pandas as pd
df_chest = pd.read_csv("../data/Chest/Data_Entry_2017.csv")
df_chest["labels-count"] = df_chest["Finding Labels"].str.split("|")
df_chest = df_chest[df_chest["labels-count"]==1]

In [ ]:
labels = df_chest["Finding Labels"].unique()
labels

In [76]:
df_chest["category"] = df_chest["Finding Labels"].str.contains("No Finding").map(lambda x:int(x))#.map(lambda x:len(x))
df_chest

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,labelscount,sick,category
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,1,0,0
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,1,1,1
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,1,0,0
5,00000003_001.png,Hernia,1,3,74,F,PA,2500,2048,0.168,0.168,NaN,1,0,0
6,00000003_002.png,Hernia,2,3,75,F,PA,2048,2500,0.168,0.168,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112114,00030801_000.png,No Finding,0,30801,39,M,PA,2500,2048,0.168,0.168,NaN,1,1,1
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN,1,1,1
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN,1,1,1
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN,1,1,1


In [ ]:
# group by the labels
df_chest_grouped = df_chest.groupby("Finding Labels")

In [85]:
import os

df_list = []
for i in range(1,13):
    dire = "../data/Chest/images_00"
    if i>=10:
        dire = dire[:-1]
    path = dire+str(i)+"/images"
    liste = os.listdir(path)
    df_cest_new = df_chest[df_chest["Image Index"].isin(liste)].copy()
    df_cest_new["path"] = path[14:]
    df_list.append(df_cest_new)


df4 = pd.concat(df_list)
df4["path"] = df4["path"] + "/" + df4["Image Index"]
df5 = df4[["path","category"]]


In [89]:
df5.to_csv("../data/Chest/allLabels.csv",index=False)

In [3]:
# read csv file
import pandas as pd
df_chest = pd.read_csv("../data/Chest/allLabels.csv")

In [1]:
import json
# importing the module

# Opening JSON file
with open("../modeltraining/modeltemplates/params.json") as json_file:
    data = json.load(json_file)

In [3]:
type(data)

dict

In [6]:
import pandas as pd
df_hh = pd.read_csv("../data/Chest/allLabels.csv")
df_sample = df_hh.groupby('category', group_keys=False).apply(lambda x: x.sample(50))
df_sample = df_sample.sample(frac=1).reset_index(drop=True)


In [10]:
df_sample.to_csv("../data/Chest/samples100.csv",index=False)

In [13]:
bla = [True,False,True]
bla*1

[True, False, True]